In [ ]:
EMNt = function(x, mu, sigma, Gamma, delta, nu, maxiter = 500, tol = 1e-8, print = FALSE){
    n = length(x)
    m = length(mu) 
    mu.next = mu 
    sigma.next = sigma 
    Gamma.next = Gamma 
    delta.next = delta 
    nu.next = nu
    u = numeric(n) 
    llk.prev = 0

for (iter in 1:maxiter){ 
    lallprobs = matrix(NA, n, m)
    for (i in 1:n){ 
        for (j in 1:m){
            if (j < m)
                lallprobs[i, j] = dnorm(x[i], mean = mu.next[j],
                sd = sigma.next[j], log = T)
            else
                lallprobs[i, j] = dtmod(x[i], mu.next[j], sigma.next[j],
                nu.next, log = T)
                }
}
fb = lalphabeta(x, mu.next, sigma.next, Gamma.next, delta.next, nu.next)
la = fb$la 
lb = fb$lb 
c = max(la[, n])
llk = c + log(sum(exp(la[, n] - c))) 
for (j in 1:m){
    for (k in 1:m)
        Gamma.next[j, k] = Gamma[j, k]*sum(exp(la[j, 1:(n-1)] +
        lallprobs[2:n, k] + lb[k, 2:n] - llk))
    if (j < m){
        mu.next[j] = exp(la[j, ] + lb[j, ] - llk)%*%x/
        sum(exp(la[j, ] + lb[j, ] - llk)) 
        sigma.next[j] = sqrt((exp(la[j, ] + lb[j, ] - llk)*
        (x - mu.next[j]))%*%(x - mu.next[j])/ sum(exp(la[j, ] + lb[j, ] - llk)))
        }

    else{
        u = (nu.next + 1)/(nu.next + (x - mu.next[j])*
        (x - mu.next[j])/sigma.next[j]^2)
        w = (exp(la[j, ] + lb[j, ] - llk)*u)%*%x
        z = exp(la[j, ] + lb[j, ] - llk)%*%u
        mu.next[j] = w/z
        w = (exp(la[j, ] + lb[j, ] - llk)*u*(x - mu.next[j]))%*%
        (x - mu.next[j]) 
        sigma.next[j] = sqrt(w/z)
        w = exp(la[j, ] + lb[j, ] - llk)%*%(log(u) - u) 
        z = sum(exp(la[j, ] + lb[j, ] - llk))
        estimator = function(nu, frac){
        -digamma(exp(nu)/2)+log(exp(nu)/2) + 1 + frac + digamma((exp(nu) + 1)/2) - log((exp(nu) + 1)/2)
        }
        nu.next = exp(uniroot(estimator, frac = w/z, lower = log(0.01), upper = log(500))$root)
        }
        }

Gamma.next = Gamma.next/apply(Gamma.next, 1, sum) 
delta.next = exp(la[, 1] + lb[, 1] - llk) 
delta.next = delta.next/sum(delta.next)
crit = llk - llk.prev
if(crit < tol){
    np = (m + 2)*m
    AIC = -2*(llk - np)
    BIC = -2*llk + np*log(n)
    return(list(mu = mu, sigma = sigma, Gamma = Gamma,
    delta = delta, nu = nu, iterations = iter, mllk = llk, AIC = AIC, BIC = BIC))}

mu = mu.next
sigma = sigma.next 
Gamma = Gamma.next 
delta = delta.next 
nu = nu.next 
llk.prev = llk 
if(print == T)
    print(paste('Iteration', iter, 'LogLik', round(llk, 4)))}
    
print(paste('No convergence after', maxiter, 'iterations')) 
return(list(mu = mu, sigma = sigma, Gamma = Gamma,
    delta = delta, nu = nu, iterations = iter, mllk = llk))}

lalphabeta = function(x, mu, sigma, Gamma, delta, nu){ 
    n = length(x)
    m = length(mu)
    lalpha = lbeta = matrix(NA, m, n) 
    P = rep(NA, m)
    for (j in 1:m){
        if (j < m)
            P[j] = dnorm(x[1], mean = mu[j], sd = sigma[j])
        else
            P[j] = dtmod(x[1], mu[j], sigma[j], nu)
    }
foo = delta*P
sumfoo = sum(foo)
lscale = log(sumfoo)
foo = foo/sumfoo
lalpha[, 1] = log(foo) + lscale 
for (i in 2:n){
    for (j in 1:m){ 
        if (j < m)
            P[j] = dnorm(x[i], mean = mu[j], sd = sigma[j]) 
        else
            P[j] = dtmod(x[i], mu[j], sigma[j], nu)
}
foo = foo%*%Gamma*P
sumfoo = sum(foo)
lscale = lscale + log(sumfoo) foo = foo/sumfoo
lalpha[, i] = log(foo) + lscale
}
lbeta[, n] = rep(0, m) 
foo = rep(1/m, m)
lscale = log(m)
for (i in (n - 1):1){
    for (j in 1:m){ 
        if (j < m)
            P[j] = dnorm(x[i + 1], mean = mu[j], sd = sigma[j]) 
        else
            P[j] = dtmod(x[i+1], mu[j], sigma[j], nu)
}
    foo = Gamma%*%(P*foo)
    lbeta[, i] = log(foo) + lscale 
    sumfoo = sum(foo)
    foo = foo/sumfoo
    lscale = lscale + log(sumfoo)
}
list(la = lalpha, lb = lbeta)

dtmod = function(x, mu = 0, sigma = 1, nu = 30, log = FALSE){ 
    den1 = try(sigma*beta(1/2, nu/2))
    num1 = try(nu^(-1/2))
    den2 = try(nu*sigma^2)
    num2 = try((x - mu)^2)
    dtmod = try(num1/den1*(1 + num2/den2)^(-1*(nu + 1)/2)) 
    if (log == TRUE)
        dtmod = log(dtmod) 
    return (dtmod)
    }